In [1]:
import requests
import json
import time
import os
import pandas as pd
import requests
from pandas.io.json import json_normalize
import csv

### Information

Input files:
None

Output files:
raw_data/all_eclipse_bugs_full.csv
raw_data/bug_dependencies.csv


In [2]:
complete_dataset_file_name = 'raw_data/all_eclipse_bugs_full.csv'
bug_dependencies_file = './raw_data/bug_dependencies.csv'

In [ ]:
user_name = 'zainab.ab.k7@gmail.com'
passw = 'wRc6BA7MHXqNeQQ'
r = requests.get('https://bugs.eclipse.org/bugs/rest/login?login='+user_name+'&password='+passw)

In [ ]:
#inititally it was a string variable - we need to convert it to a dictionary
token_var = json.loads(r.text)
token_var = token_var['token']
token_var

In [3]:
#https://bugzilla.readthedocs.io/en/5.0/api/core/v1/bug.html#search-bugs
products = ['Platform', 'JDT', 'PDE','E4','Incubator','Equinox']


In [ ]:
for product in products:
    offset = 0
    limit = 1000

    while True:
        target_file = 'raw_data/json/eclipse_bugs_product='+str(product)+'_offset='+str(offset)+'.json'
        if not os.path.exists(target_file):
            link = 'https://bugs.eclipse.org/bugs/rest/bug?product='+product+'&limit='+str(limit)+'&offset='+str(offset)+'&token='+token_var
            ff = requests.get(link)
            plat_bugs = ff.text
            jsonarray = json.loads(plat_bugs)
            jsonarray = jsonarray['bugs']
            if len(str(jsonarray))==0 or len(jsonarray)==0:
                break

            with open(target_file, 'w') as outfile:
                json.dump(jsonarray, outfile)

        offset+=limit
    time.sleep(1)

In [4]:
#Get the bugs of all products without any version field constraints
if os.path.exists(complete_dataset_file_name):
    df_all = pd.read_csv(complete_dataset_file_name,index_col=False)
else:
    df_all = pd.DataFrame()
    
    df_depends_on = pd.DataFrame()
    
    path_to_json = 'raw_data/json'

    results = pd.DataFrame()
  
    for product in products:
        print(product)
        json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
        json_files = [x for x in json_files if '_product='+product in x and '_offset'  in x]


        for file in json_files:
            with open('raw_data/json/'+file) as f:
                data = json.load(f)
            
            norm = json_normalize(data)
            bug_depends_on = norm[['id','depends_on']]
            bug_depends_on  = (
                bug_depends_on
                .depends_on.apply(pd.Series)
                .merge(bug_depends_on, left_index = True, right_index = True)
                .drop(["depends_on"], axis = 1)
                .melt(id_vars = ['id'], value_name = "depends_on")
                .drop(['variable'],axis=1)
                .dropna()
            )
            
            df_depends_on = df_depends_on.append(bug_depends_on,ignore_index=True)

            df_all = df_all.append(norm,ignore_index=True)
            
    #export the files
    df_all.to_csv(complete_dataset_file_name,index=False,quoting=csv.QUOTE_NONNUMERIC)
    df_depends_on.to_csv(bug_dependencies_file,index=False,quoting=csv.QUOTE_NONNUMERIC)

Platform


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


JDT
PDE
E4
Incubator
Equinox
